In [25]:
import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

In [26]:
# Load .env file
load_dotenv()

True

In [28]:
# Check if the environment variable is set
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("The environment variable 'OPENAI_API_KEY' is not set.")